
# 📈 E-commerce Sales Dashboard & Retention Analysis

This project analyzes open-source e-commerce transaction data to uncover trends in customer purchase behavior, retention, and product performance. Tools used: **SQL (via SQLite), Python (Pandas, Seaborn)**, and **Power BI** for visualization.

**Goals:**
- Segment customers by lifecycle stage (new vs. repeat)
- Analyze retention and customer value
- Visualize trends in sales and product performance

**Dataset**: [`Online Retail Dataset`](https://archive.ics.uci.edu/ml/datasets/Online+Retail)


In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
sns.set(style='whitegrid')


In [ ]:

# Download the dataset first manually from the UCI ML repo or Kaggle
# Expected file: 'OnlineRetail.csv'
df = pd.read_excel('OnlineRetail.xlsx')
df.head()


In [ ]:

# Remove rows with missing CustomerID and filter out cancellations
df.dropna(subset=['CustomerID'], inplace=True)
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype(str)
df.head()


In [ ]:

df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
monthly_revenue = df.groupby('InvoiceMonth')['TotalPrice'].sum().reset_index()
monthly_revenue['InvoiceMonth'] = monthly_revenue['InvoiceMonth'].astype(str)

plt.figure(figsize=(12,5))
sns.lineplot(data=monthly_revenue, x='InvoiceMonth', y='TotalPrice', marker='o')
plt.title("Monthly Revenue Trend")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

min_purchase = df.groupby('CustomerID')['InvoiceDate'].min().reset_index()
min_purchase.columns = ['CustomerID', 'FirstPurchaseDate']
df = df.merge(min_purchase, on='CustomerID')

df['UserType'] = df.apply(lambda row: 'New' if row['InvoiceDate'] == row['FirstPurchaseDate'] else 'Returning', axis=1)

user_segmentation = df.groupby(['InvoiceMonth', 'UserType'])['CustomerID'].nunique().reset_index()
user_segmentation['InvoiceMonth'] = user_segmentation['InvoiceMonth'].astype(str)

plt.figure(figsize=(12,5))
sns.barplot(data=user_segmentation, x='InvoiceMonth', y='CustomerID', hue='UserType')
plt.title("New vs Returning Customers by Month")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

df['CohortMonth'] = df.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

def get_month_diff(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

df['CohortIndex'] = df.apply(lambda row: get_month_diff(row['InvoiceDate'], row['CohortMonth'].to_timestamp()), axis=1)

cohort_data = df.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().reset_index()
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)

plt.figure(figsize=(12, 6))
sns.heatmap(retention, annot=True, fmt='.0%', cmap='Blues')
plt.title('Cohort Analysis - Retention Rates')
plt.ylabel('Cohort Month')
plt.xlabel('Months Since First Purchase')
plt.tight_layout()
plt.show()
